In [2]:
import os, sys
import numpy as np
import ants
import argparse
import shutil
import logging
import subprocess
import tqdm
import matplotlib.pyplot as plt 
import pandas as pd

### Testing must inclide check on:
1. Nan and Infifnite values
2. int in labels
3. Different to parent folder
4. Equal orientation to parent folder
5. Visual inspection on one modality

In [13]:
## check for nans
for patient in tqdm.tqdm(os.listdir('/anvar/public_datasets/preproc_study/bgpd/4a_resamp/')):
    if patient != 'logging.txt':
        
        for modality in ['T1.nii.gz','T2.nii.gz','FLAIR.nii.gz','CT1.nii.gz','mask_GTV_FLAIR.nii.gz']:
            img = ants.image_read('/anvar/public_datasets/preproc_study/bgpd/4a_resamp/{}/{}'.format(patient, modality))
            if not img.sum() > 0:
                print(patient,modality, 'Amount of nans:', np.shape(np.argwhere(np.isnan(img.numpy()))))

100%|██████████| 181/181 [04:13<00:00,  1.40s/it]


In [26]:
## nans and wrong labels
label = np.asarray([0,1])

for patient in tqdm.tqdm(os.listdir('/anvar/public_datasets/preproc_study/bgpd/4a_resamp/')):
    if patient != 'logging.txt':
        for modality in ['T1.nii.gz','T2.nii.gz','FLAIR.nii.gz','CT1.nii.gz','mask_GTV_FLAIR.nii.gz']:
            img = ants.image_read('/anvar/public_datasets/preproc_study/bgpd/4a_resamp/{}/{}'.format(patient, modality))
            if not img.sum() > 0:
                print(patient,modality, 'Amount of nans:', np.shape(np.argwhere(np.isnan(img.numpy()))))
            if modality == 'mask_GTV_FLAIR.nii.gz':
                ## very ugly check if there are needed values in mask
                if not (np.unique(img.numpy()) == label).sum() == 2:
                    print('Assert label', patient, np.unique(img.numpy()) )

100%|██████████| 181/181 [05:03<00:00,  1.68s/it]


In [5]:
os.listdir('/anvar/public_datasets/preproc_study/bgpd/')

['3a_atlas',
 'orig',
 '3b_n4',
 '5_ss_shared',
 '2a_interp',
 '3d_susan',
 '4a_resamp',
 '1_reg',
 '6_hist']

In [11]:
import tqdm
path = '4a_resamp'

print('/home/kate/projects/qc_preprocessing/{}_bgpd/'.format(path))
os.makedirs('/home/kate/projects/qc_preprocessing/{}_bgpd/'.format(path), exist_ok = True)

def plot_qc(image, mask, patient_id):
    fig, axs = plt.subplots(3,3,figsize=(12,12))
    idx = np.argsort(mask.sum(axis=(0,1)))[-3:]
    for i in range(3):
        axs[i, 0].imshow(image[..., idx[i]], cmap='gray')
        axs[i, 1].imshow(mask[..., idx[i]], cmap='gray')
        axs[i, 2].imshow(image[..., idx[i]]+200*mask[..., idx[i]], cmap='gray')
    axs[0,0].set_title(patient_id)
    plt.tight_layout()
    plt.savefig(f'/home/kate/projects/qc_preprocessing/{path}_bgpd/{patient_id}.png', dpi=300)
    plt.plot()
    plt.close(fig)

output = '/anvar/public_datasets/preproc_study/bgpd/{}/'.format(path)

for patient in tqdm.tqdm(os.listdir(output)):
    try:
        im = ants.image_read(output + patient +'/CT1.nii.gz')
        mask = ants.image_read(output + patient +'/mask_GTV_FLAIR.nii.gz')
        plot_qc(im, mask, patient_id=str(patient))
    except Exception as e:
        print(e, patient)

  0%|          | 0/181 [00:00<?, ?it/s]

/home/kate/projects/qc_preprocessing/4a_resamp_bgpd/


 92%|█████████▏| 166/181 [07:40<00:38,  2.54s/it]

File /anvar/public_datasets/preproc_study/bgpd/4a_resamp/logging.txt/CT1.nii.gz does not exist! logging.txt


100%|██████████| 181/181 [08:18<00:00,  2.76s/it]
